# Rebuilding FPI Scraper 2024

Because they reformatted the page or something.

`powers = anysunday.loaders.espn_powers()`

```
---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
Cell In[10], line 1
----> 1 powers = anysunday.loaders.espn_powers()

File ~/Documents/Code/nfl-survival-picks/anysunday/loaders.py:62, in espn_powers()
     59 data_table = selector.xpath("//div[@class='Table__Scroller']/table")
     61 tables = selector.xpath("//div[contains(concat(' ', @class, ' '), ' league-nfl ')]//table")
---> 62 assert len(tables) == 2
     63 name_table, data_table = tables
     65 club_table = {}

AssertionError:
```

In [8]:
import requests
import parsel

In [9]:
resp = requests.get("https://www.espn.com/nfl/fpi")
resp

<Response [403]>

In [10]:
print(resp.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>403 ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
Request blocked.
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>
Generated by cloudfront (CloudFront)
Request ID: a0Bkc2gyoZHpsgTL5W2wLQe4CdjAaGVGZpKNE3LFv8wn4YjRhz358Q==
</PRE>
<ADDRESS>
</ADDRESS>
</BODY></HTML>


Meh.

In [11]:
resp = requests.get("https://www.espn.com/nfl/fpi", headers={"User-Agent": "Mozilla/5.0"})
resp

<Response [200]>

Yay. That was easy.

In [14]:
from anysunday.loaders import URL_ESPN_FPI_PAGE, NUM_TEAMS

def espn_powers():
    """
    Loads ESPN power ranks and returns them as a TEAM: POWER mapping
    """
    resp = requests.get(URL_ESPN_FPI_PAGE, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    data = resp.text
    selector = parsel.Selector(data)
    data_table = selector.xpath("//div[@class='Table__Scroller']/table")

    tables = selector.xpath("//div[contains(concat(' ', @class, ' '), ' league-nfl ')]//table")
    assert len(tables) == 2
    name_table, data_table = tables

    club_table = {}
    for club in name_table.xpath("descendant::*[@data-idx]"):
        club_idx = club.xpath("@data-idx").extract_first()
        club_url = club.xpath("descendant::*[@data-clubhouse-uid]/@href")
        url = club_url.extract_first()
        parts = url.split("/")
        assert parts[4] == "name"
        club_table[parts[5].strip().upper()] = int(club_idx)

    assert len(club_table) == NUM_TEAMS

    powers = {}
    for club, idx in club_table.items():
        xq = f"tbody/*[@data-idx='{idx}']"
        r = data_table.xpath(xq)
        cols = r.xpath("td/div/text()")
        pwr = float(cols[1].extract())
        powers[club] = pwr

    assert set(powers) == set(club_table)

    return powers

In [15]:
espn_powers()

{'SF': 5.3,
 'KC': 4.6,
 'DET': 4.2,
 'BAL': 3.6,
 'PHI': 3.6,
 'BUF': 3.2,
 'HOU': 2.7,
 'GB': 2.7,
 'DAL': 2.6,
 'CIN': 2.5,
 'MIA': 2.2,
 'NYJ': 1.7,
 'LAR': 1.3,
 'CLE': 0.5,
 'CHI': 0.4,
 'ATL': 0.3,
 'JAX': 0.0,
 'SEA': -0.2,
 'LAC': -0.2,
 'PIT': -0.4,
 'IND': -0.9,
 'TB': -1.4,
 'ARI': -1.9,
 'MIN': -2.1,
 'NO': -2.4,
 'LV': -3.2,
 'TEN': -3.3,
 'NYG': -4.1,
 'WSH': -4.1,
 'DEN': -4.7,
 'NE': -5.6,
 'CAR': -6.1}